In [1]:
import numpy as np
seed = 1234
np.random.seed(seed)

from sklearn.datasets import load_wine
x, y = load_wine(return_X_y=True)

x = x[:59+71]
y = y[:59+71]

from sklearn.model_selection import train_test_split
x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size=0.9)

from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

x_test = scaler.transform(x_test)
x_test = np.clip(x_test, 0, 1)

In [9]:
import pennylane as qml

nqubits = 4
dev = qml.device("default.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(
        a, wires=range(nqubits), pad_with=0, normalize=True)
    qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(nqubits), pad_with=0, normalize=True))
    return qml.probs(wires=range(nqubits))

from sklearn.svm import SVC
def qkernel(A, B):
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

svm = SVC(kernel=qkernel).fit(x_tr, y_tr)

from sklearn.metrics import accuracy_score
print(accuracy_score(svm.predict(x_test), y_test))


0.9230769230769231


## SRJ - Kernel based method 

In [1]:
import numpy as np
#import torch
#from torch.nn.functional import relu

from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers

import matplotlib.pyplot as plt

np.random.seed(42)

In [2]:
X, y = load_iris(return_X_y=True)   #Loadind the iris dataset

# pick inputs and labels from the first two classes only,
# corresponding to the first 100 samples
X = X[:100]  #Taking the first 100 samples
y = y[:100] #Taking the first 100 labels
#print('x values beofre scaling:', X)  #X features y labels

# scaling the inputs is important since the embedding we use is periodic
scaler = StandardScaler().fit(X)  #StandardScaler standardizes features by removing the mean and scaling to unit variance.
X_scaled = scaler.transform(X)  #The transform(X) method applies the scaling to convert the original data into a scaled version. Specifically, it subtracts the mean and divides by the standard deviation feature-wise
print('x values after scaling:', X_scaled)

# scaling the labels to -1, 1 is important for the SVM and the
# definition of a hinge loss
y_scaled = 2 * (y - 0.5) #Effectively, it maps values originally in the range [1] to {-1, 1} by subtracting 0.5 and multiplying by 2.

print('y scaled',y_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, train_size=0.25, random_state=42)
print('X_train:', len(X_test))

x values after scaling: [[-5.81065904e-01  8.41837140e-01 -1.01297765e+00 -1.04211089e+00]
 [-8.94308978e-01 -2.07835104e-01 -1.01297765e+00 -1.04211089e+00]
 [-1.20755205e+00  2.12033793e-01 -1.08231219e+00 -1.04211089e+00]
 [-1.36417359e+00  2.09934449e-03 -9.43643106e-01 -1.04211089e+00]
 [-7.37687441e-01  1.05177159e+00 -1.01297765e+00 -1.04211089e+00]
 [-1.11201292e-01  1.68157493e+00 -8.04974023e-01 -6.86441647e-01]
 [-1.36417359e+00  6.31902691e-01 -1.01297765e+00 -8.64276271e-01]
 [-7.37687441e-01  6.31902691e-01 -9.43643106e-01 -1.04211089e+00]
 [-1.67741667e+00 -4.17769553e-01 -1.01297765e+00 -1.04211089e+00]
 [-8.94308978e-01  2.09934449e-03 -9.43643106e-01 -1.21994552e+00]
 [-1.11201292e-01  1.26170604e+00 -9.43643106e-01 -1.04211089e+00]
 [-1.05093052e+00  6.31902691e-01 -8.74308565e-01 -1.04211089e+00]
 [-1.05093052e+00 -2.07835104e-01 -1.01297765e+00 -1.21994552e+00]
 [-1.83403820e+00 -2.07835104e-01 -1.22098127e+00 -1.21994552e+00]
 [ 5.15284858e-01  1.89150938e+00 -1.1

In [8]:
print(y_train[0])
print(y_train[0].shape)

print(len(X_test))

-1.0
()
25


## Features - x values - input 

In [3]:
n_qubits = len(X_train[-1])
n_qubits

4

In [4]:
dev_kernel = qml.device("default.qubit", wires=n_qubits)

projector = np.zeros((2 ** n_qubits, 2 ** n_qubits))  # Creating a square matrix of size 2^n_qubits
projector[0, 0] = 1 #Complete overlap with its same state

@qml.qnode(dev_kernel)
def kernel(x1, x2):
    """The quantum kernel."""
    AngleEmbedding(x1, wires=range(n_qubits))  #x1 is feature vector, Rx gate is applied
    qml.adjoint(AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.expval(qml.Hermitian(projector, wires=range(n_qubits)))

In [34]:
kernel(X_train[0], X_train[0])

np.float64(0.9999999999999998)

In [31]:
len(X_train)

75

In [14]:
## Prepare state1

@qml.qnode(dev_kernel)
def state1(x1):
    qml.AngleEmbedding(x1, wires=range(n_qubits))  #x1 is feature vector, Rx gate is applied
    return qml.state()


def state2(x2):
    k = qml.adjoint(qml.AngleEmbedding(x2, wires=range(n_qubits)))
    return k
    

def smatrix(A, B):
    total_mat = np.zeros((len(A), len(B)))  #Creating a square matrix of size 2^n_qubits
    print('The shape of the total matrix:', total_mat.shape)
    for i in range(len(A)):
        for j in range(len(B)):
            print(f'The value of i and j: {i}, {j}')
            right_op = state1(x1=A[i])
            left_op = state2(x2=B[j])
            total_mat[i,j] = np.abs(np.dot(left_op, right_op))**2
            print(f'Value of total_mat[{i}, {j}]: {total_mat[i, j]}')
    return total_mat

svm = SVC(kernel=smatrix).fit(X_train, y_train)
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)
print("Accuracy:", accuracy_score(predictions, y_test))


The shape of the total matrix: (75, 75)
The value of i and j: 0, 0


TypeError: bad operand type for abs(): 'SProd'

In [5]:
def kernel_matrix(A, B):
    """Compute the matrix whose entries are the kernel
       evaluated on pairwise data from sets A and B."""

    return np.array([[kernel(a, b) for b in B] for a in A])




svm = SVC(kernel=kernel_matrix).fit(X_train, y_train)

predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)
print("Accuracy:", accuracy_score(predictions, y_test))


Accuracy: 1.0


In [12]:
@qml.qnode(dev_kernel)
def overlap_kernel(x1, x2):
    qml.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.AngleEmbedding)(x2, wires=range(n_qubits))
    return qml.state()

def matrix_fn(A, B):
    kernel_matrix = np.zeros((len(A), len(B)))
    for i in range(len(A)):
        for j in range(len(B)):
            psi = overlap_kernel(A[i], B[j])  # returns statevector
            prob_0 = np.abs(psi[0])**2  # Probability amplitude for |0...0>
            kernel_matrix[i, j] = prob_0
    #print('Kernel matrix shape:', kernel_matrix)  
    return kernel_matrix
svm = SVC(kernel=matrix_fn).fit(X_train, y_train)
predictions = svm.predict(X_test)
accuracy_score(predictions, y_test)
print("Accuracy:", accuracy_score(predictions, y_test))

Accuracy: 1.0
